In [3]:
import bs4
import re
import requests
import pandas as pd

# results
df = pd.DataFrame(columns = ['link', 'title','author', 'cc'])

# arquivo baixado de: http://books.uc.pt/catalogo?all
text_file = open("../data/2023/celia/celia.html", "r")
data = text_file.read()
text_file.close()

In [2]:
soup = bs4.BeautifulSoup(data)

# links para livros
links = soup.find_all("a", string = "LER +")

for link in links:
    href = link.get('href')
    url = 'http://books.uc.pt/' + href
    res = requests.get(url)
    book = bs4.BeautifulSoup(res.text)
    
    # pega titulos
    title = book.find('h1').text
    
    # pega autores
    author = ''
    if book.find('a', {'href': re.compile('author\?author')}) is not None:
        author = book.find('a', {'href': re.compile('author\?author')}).text 
    
    # verifica se é creative commons
    has_cc = book.find('svg', {'id': 'CC'})
    cc = 0
    if has_cc is not None:
        cc = 1
    
    # Monta dicionário com todos campos coletados
    dict = {'link' : url, 
            'title' : title,
            'author': author,
            'cc' : cc}
    
    # Adiciona no dataframe
    row = pd.DataFrame([dict])
    df = pd.concat([df, row])
    
df.to_csv('/tmp/celia.csv', index=False)

In [100]:
import pandas as pd
df = pd.read_csv ('../data/2023/celia/result.csv')

# Total de livros sem CC
sum(df['cc'] == 0.0) 

470

In [101]:
# total de livros com CC
df['cc'].sum()

824.0

In [102]:
len(df)

1295

In [80]:
def coleta(url):
    res = requests.get(url)
    book = bs4.BeautifulSoup(res.text)

    # Título 
    title = book.find('h1').text

    # bloquinho direita
    bloquinho = book.find("p", {"class": ["small"] }).text
    linhas = bloquinho.splitlines()
    edicao = linhas[0]
    isbn = linhas[1]
    eisbn = linhas[2]
    doi = linhas[3]
    serie = linhas[4]
    data = linhas[6]

    # Palavras Chave
    keywords = book.find_all("p", {"class": ["small"] })[1].text
    keywords = keywords.replace("Palavras-Chaves", "")

    # Autores
    #authors = book.find('a', {'href': re.compile('author\?author')}).text 
    authors = book.find_all('a', {'href': re.compile('author\?author')})
    authors_string = ''
    afiliacao_string = ''
    for author in authors:
        authors_string = authors_string + ';' + author.text
        url_author = author.get('href')
        
        # Visitando a paǵina do autor para pegar a afiliação
        res_author = requests.get('http://books.uc.pt/' + url_author)
        author_page = bs4.BeautifulSoup(res_author.text)
        afiliacao = author_page.find('span', {"class": ["text-md","font-medium"] }).text
        afiliacao_string = afiliacao_string + ';' + afiliacao        

    # 0 não é CC e 1 é CC
    has_cc = book.find('svg', {'id': 'CC'})
    cc = 0 

    doi =  doi.replace("DOI: ", "")
    url_doi = requests.head('https://doi.org/'+doi, allow_redirects=True)
    url_redirected = url_doi.url
    if 'monographs' in url_redirected:
        print(url_redirected)
    # PARAMOS AQUI: pegar a informação se tem google play e Amazon    
    #teste = requests.get(, allow_redirects=True)
    #xx = bs4.BeautifulSoup(teste)
    #yy = xx.find('a', {"class": ["cmp_download_link"] })
    #print(yy)

    
    return {
        'tilulo':    title,
        'autores':   authors_string,
        'afiliacao': afiliacao_string,
        'edicao': edicao,
        'isbn': isbn,
        'eisbn,': eisbn,
        'doi': doi,
        'serie': serie, 
        'data': data,
        'cc': cc,
        'keywords': keywords
    }

url = 'http://books.uc.pt/book?book=1324'
print(coleta(url))

#print(title)
#print(authors)

http://monographs.uc.pt/iuc/catalog/book/411
{'tilulo': 'Deficiência em Portugal: Lugares, corpos e lutas', 'autores': ';Fernando Fontes;Bruno Sena Martins', 'afiliacao': ';Centro de Estudos Sociais da Universidade de Coimbra;Centro de Estudos Sociais da Universidade de Coimbra', 'edicao': '1.ª Edição', 'isbn': 'ISBN: 978-989-26-2423-5', 'eisbn,': 'eISBN: 978-989-26-2424-2', 'doi': '10.14195/978-989-26-2424-2', 'serie': 'Série: Coleção CES', 'data': 'Data: Dezembro, 2023', 'cc': 0, 'keywords': 'Deficiência Portugal Direitos Humanos Lutas sociais Experiência   \n\t  '}
